In [0]:
from pyspark.sql import functions as F
from delta.tables import *

# Origens
df_chamado = spark.read.table("v_credit.silver.tb_chamado")
df_log = spark.read.table("v_credit.silver.tb_chamado_log")
df_custo = spark.read.table("v_credit.silver.tb_custo_chamado")
df_pesquisa = spark.read.table("v_credit.silver.tb_pesquisa")

# Destino
TABELA_DESTINO = "v_credit.gold.ft_atendimento_geral"

In [0]:
# Join das Tabelas Silver
df_join = (
    df_chamado.alias("c")
    .join(df_log.alias("l"), on="cd_chamado", how="left")
    .join(df_custo.alias("cust"), on="cd_chamado", how="left")
    .join(df_pesquisa.alias("p"), on="cd_chamado", how="left")
)

# Transformação
df_gold = df_join.select(
    # PK Hash Composta
    F.md5(F.concat(
        F.coalesce(F.col("c.cd_cliente").cast("string"), F.lit("0")),
        F.coalesce(F.col("c.cd_motivo").cast("string"), F.lit("0")),
        F.coalesce(F.col("c.cd_canal").cast("string"), F.lit("0")),
        F.coalesce(F.col("c.cd_atendente").cast("string"), F.lit("0")),
        F.coalesce(F.col("c.cd_chamado").cast("string"), F.lit("0"))
    )).alias("pk_fato_atendimento"),
    
    # FKs para Dimensões
    F.col("c.cd_cliente").cast("string"),
    F.col("c.cd_motivo"),
    F.col("c.cd_canal"),
    F.col("c.cd_atendente"),
    
    # Degenerate Dim
    F.col("c.cd_chamado").cast("bigint").alias("cd_chamado"),
    
    # Tempo
    F.to_date(F.coalesce(F.col("l.dh_inicio"), F.col("c.dt_ingestion"))).alias("dt_referencia"),
    F.hour(F.coalesce(F.col("l.dh_inicio"), F.col("c.dt_ingestion"))).cast("smallint").alias("nu_hora_dia"),
    
    # Métricas e Flags
    F.coalesce(F.col("c.tm_espera"), F.lit(0)).cast("bigint").alias("val_tempo_espera"),
    F.coalesce(F.col("c.tm_duracao"), F.lit(0)).cast("bigint").alias("val_tempo_atendimento"),
    
    # Custo (Decimal 12,10)
    F.coalesce(F.col("cust.vl_custo"), F.lit(0)).cast("decimal(12,10)").alias("val_custo"),
    
    F.col("p.nu_nota").cast("smallint").alias("val_nota_csat"),
    
    F.when(F.col("c.st_resolvido") == True, 1).otherwise(0).cast("smallint").alias("st_resolvido"),
    
    F.when(
        (F.col("c.tm_espera") > 300) | (F.col("p.nu_nota") <= 2), 1
    ).otherwise(0).cast("smallint").alias("fl_experiencia_negativa"),
    
    # CORREÇÃO AMBIGUIDADE: Especificando que vem de 'c' (chamado)
    F.col("c.dt_ingestion"),
    F.col("c.dc_origem")
)



DeltaTable.forName(spark, TABELA_DESTINO).alias("t").merge(
    df_gold.alias("s"), "t.pk_fato_atendimento = s.pk_fato_atendimento"
).whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()

print(f"✅ Fato Atendimento Geral carregada (Linhagem preservada)!")